# About this implementation


Might make most sense to continue to use sklearn for the test/train split because i dont know how it applies the random state if there is a difference in pytorch, and I want to compare my results between implementations.

### Results
F1 Score (y=0) 0.95

F1 Score (y=1) 0.97

Currently using the Adam Optimizer with weight decay 1e-5 (solver provides l2 penalty)


### Step 1: import the dataset and prepare the data

In [1]:
# use sklearn to import the dataset
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# load dataset
dataset = datasets.load_breast_cancer()

In [3]:
dataset.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [4]:
dataset.target_names

array(['malignant', 'benign'], dtype='<U9')

In [5]:
# split test train
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=42)

In [6]:
# scale the X values of the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # 'fit' stats.. mean, etc. then 'transform'
X_test = scaler.transform(X_test) # uses previous 'fit' for this test set, and 'transforms'

In [68]:
#
# move ndarray to tensors
#
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1,1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1,1)

## Step 2: define the model

do this with torch

In [63]:
import torch

import torch.nn.functional as F # if want functional API
import torch.nn as nn           # if want module API
import torch.optim as optim

In [64]:
class LogisticRegressionModel(torch.nn.Module):
    
    def __init__(self, num_features):
        
        super(LogisticRegressionModel, self).__init__()         # calls init function of nn.module
        self.linear_combination = nn.Linear(num_features, 1) # 1 output node
        
        # explicitly init weights as zero 
        # torch.nn.Linear by default set them to small, random numbers
        self.linear_combination.weight.detach().zero_()
        self.linear_combination.bias.detach().zero_()
        
        
    def forward(self, x):
        
        out = self.linear_combination(x)               # weighted sum
        posterior_probabilities = torch.sigmoid(out)   # p(x) = a = sigmoid(x) = h(x)
        
        return posterior_probabilities

## Step 3: Loop for train the model

$$ \ell = - \sum_{i=1}^{n} y_i \; log y_i + (1-y_i)log(1-y_i) $$


In [70]:
torch.manual_seed(42)

In [72]:
X_train.shape
num_features = X_train.shape[1]
num_features

30

In [105]:
model = LogisticRegressionModel(num_features)

# Loss and solver
criterion = nn.BCELoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001) # try a diff one w regularization

# adam optimizer is a type of l2 regulizer, uses 'weight decay' and 'dropout'
# which method is popular for neural network architectures
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Adjust weight_decay as needed

In [107]:
# Main Loop

num_epochs = 100
for epoch in range(num_epochs):

    # 1. forward pass, predict
    posterior_probabilities = model(X_train_tensor)
    loss = criterion(posterior_probabilities, y_train_tensor)
    
    # 2. backward, compute gradients
    optimizer.zero_grad() # zero the gradient before you start next run
    loss.backward()

    # 3. update weights
    optimizer.step()

    
    # Logging
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        
print('\n Model params:')
print('\t Weights: %s' % model.linear_combination.weight)
print('\t Bias: %s' % model.linear_combination.bias)

Epoch [10/100], Loss: 0.6351
Epoch [20/100], Loss: 0.5786
Epoch [30/100], Loss: 0.5302
Epoch [40/100], Loss: 0.4891
Epoch [50/100], Loss: 0.4541
Epoch [60/100], Loss: 0.4243
Epoch [70/100], Loss: 0.3986
Epoch [80/100], Loss: 0.3762
Epoch [90/100], Loss: 0.3567
Epoch [100/100], Loss: 0.3394

 Model params:
	 Weights: Parameter containing:
tensor([[-0.0833, -0.0855, -0.0830, -0.0830, -0.0802, -0.0784, -0.0803, -0.0823,
         -0.0784,  0.1137, -0.0823,  0.0136, -0.0813, -0.0817,  0.0766, -0.0633,
         -0.0573, -0.0755,  0.0833,  0.0710, -0.0835, -0.0860, -0.0831, -0.0831,
         -0.0837, -0.0798, -0.0803, -0.0822, -0.0840, -0.0771]],
       requires_grad=True)
	 Bias: Parameter containing:
tensor([0.0946], requires_grad=True)


## Step 4: Test

In [108]:
# set the model into eval mode
model.eval()

LogisticRegressionModel(
  (linear_combination): Linear(in_features=30, out_features=1, bias=True)
)

In [109]:
# predict but using no_grad
with torch.no_grad():
    y_pred_posterior = model(X_test_tensor)
    y_pred = torch.round(y_pred_posterior)     # convert decimal to integer for class label

## Step 5: Evaluate the model

In [110]:
from sklearn.metrics import classification_report

In [111]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        43
           1       0.97      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114

